In [3]:
import warnings
warnings.filterwarnings('ignore')

5. Run the cpu_gpu.py test file:

Adapted from https://github.com/philferriere/dlwin

additional references:
 - https://stackoverflow.com/questions/42728877/using-pymc3-on-windows-10-theano-cannot-import-name-floatx?noredirect=1#comment72639271_42728877
 - https://stackoverflow.com/a/44503723/2123721
 - http://deeplearning.net/software/libgpuarray/installation.html#windows-specific-instructions

1. Install Anaconda for windows x64 with python 3.6: https://www.anaconda.com/download/
2. Define the following variable and update PATH as shown here:
    Define sysenv variable PYTHON_HOME with the value C:\<ANACONDA_DIRECTORY>\
    Add %PYTHON_HOME%, %PYTHON_HOME%\Scripts, and %PYTHON_HOME%\Library\bin to PATH
2. Open Anaconda prompt and with a given environment name:

conda create --yes -n my_custom_py36_env python=3.6 anaconda numpy scipy mkl-service m2w64-toolchain libpython jupyter

activate my_custom_py36_env

4. Install Theano using rcdonnelly's channel

conda install -c rdonnelly Theano

4 (OPTION 1). Install pygpu and theano individually from the conda or conda-forge channels

conda install pygpu==0.6.2 nose theano

conda install -c conda-forge theano pygpu

5. Run the cpu_gpu.py test file:

'''from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')
'''